In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
import requests
from urllib.parse import urlparse
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
def bon_bun_allocator(x):
    splitted = x.split('-')
    return int(splitted[0])

def bu_bun_allocator(x):
    splitted = x.split('-')
    if len(splitted) == 2:
        return int(splitted[1])
    elif len(splitted) == 1:
        return 0
    else:
        print('error')
        return

In [3]:
def land_plans_full_address_allocator(df):    
    bonbun = df['지번'].apply(bon_bun_allocator)
    bubun = df['지번'].apply(bu_bun_allocator)
    
    df['지번주소'] = df['법정동명'] + ' ' + bonbun.apply(str) + '-' + bubun.apply(str)
    
    return df

In [4]:
def land_plan_unstack(df):
    df = df.set_index(['지번주소', '대장구분명', '년', '용도지역지구명']).unstack(-1).reset_index()
    
    return df

In [5]:
def create_land_plans_df():
    df = pd.read_csv('./토지이용계획정보/AL_11_D155_20201107.csv', encoding='euc-kr').dropna().drop_duplicates()
    df['년'] = df['등록일자'].str.split('-').apply(lambda x: x[0]).astype('int')
    
    df = land_plans_full_address_allocator(df)
    
    simple_cols = ['대장구분명', '저촉여부', '용도지역지구명', '년', '지번주소']
    
    df = df[simple_cols]
    
    df = df.drop_duplicates(subset=['대장구분명', '용도지역지구명', '년', '지번주소'], keep='first').reset_index(drop=True)
    
    df = land_plan_unstack(df)
    
    correct_cols = []
    for i in range(len(df.columns)):
        if (i == 0)|(i == 1)|(i == 2):
            correct_cols.append(df.columns[i][0])
        else:
            correct_cols.append(df.columns[i][1])
    
    df.columns = correct_cols
    
    imya_df = df[df['대장구분명'] == '임야대장']
    toji_df = df[df['대장구분명'] == '토지대장']
    both_df = imya_df[imya_df['지번주소'].isin(toji_df['지번주소'].unique())]
    df.drop(both_df.index, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df

In [6]:
land_plans_df = create_land_plans_df()

In [7]:
# NaN 어떻게 채울 것인지, 사이사이 비어있는 년도들은 어떻게 채울 것인지.

In [8]:
main_data_df = pd.read_csv('main_data_ver2.1.csv')

In [9]:
selected_land_plans_df = land_plans_df[land_plans_df['지번주소'].isin(main_data_df['지번주소'].unique())]

In [10]:
del land_plans_df, main_data_df

In [11]:
def create_complete_land_plans_df_iteratively(df):
    import numpy as np
    import pandas as pd
    import gc
    
    df = df.copy()
    
    dfs_list = []
    count = 0
    for addr in df['지번주소'].unique():
        if (count%10000) == 0:
            gc.collect()
        
        addr_df = df[df['지번주소'] == addr]
        landtype = addr_df['대장구분명'].iloc[0]
        years = [i for i in range(2006, 2021)]
        [years.remove(y) for y in addr_df['년'].unique()]
        min_year = addr_df['년'].min()

        temp_dict = {}
        for col in df.columns:
            if col == '지번주소':
                temp_dict[col] = addr
            elif col == '대장구분명':
                temp_dict[col] = landtype
            elif col == '년':
                temp_dict[col] = years
            else:
                temp_dict[col] = np.nan

        add_df = pd.DataFrame(temp_dict)

        mini_concat_df = pd.concat([addr_df, add_df]).sort_values(['년']).fillna(method='ffill')
        
        min_year_row = mini_concat_df.loc[[mini_concat_df.index[min_year-2006]],:]
        
        min_year_concat_df = pd.concat([min_year_row for i in range(min_year-2006)])
        
        mini_concat_df.iloc[[i for i in range(min_year-2006)], 3:] = min_year_concat_df.iloc[:, 3:]
        
        mini_concat_df = mini_concat_df.fillna('해당없음')
        
        dfs_list.append(mini_concat_df)
        
        count += 1

    concat_df = pd.concat(dfs_list)
    
    return concat_df

In [13]:
def parallelize(df, func, num_processors=12):
    #data_split = np.array_split(data, num_processors)
    
    data_split = []
    unique_addrs = df['지번주소'].unique()
    divided = int(len(unique_addrs) / num_processors)
    for i in range(num_processors):
        if i < (num_processors - 1):
            picked_addrs = unique_addrs[divided*i:divided*(i+1)]
        else:
            picked_addrs = unique_addrs[divided*i:]
        data_split.append(df[df['지번주소'].isin(picked_addrs)])
            
    pool = mp.Pool(num_processors)
    concat_df = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return concat_df

In [14]:
%%time
completed_df = parallelize(selected_land_plans_df, create_complete_land_plans_df_iteratively)
print(completed_df.shape)
completed_df.head()

(1609320, 332)
Wall time: 40min 7s


,지번주소,대장구분명,년,(한강)오염행위 제한지역,(한강)폐기물매립시설 설치제한지역,가로구역별 최고높이 제한지역,가스공급설비,가스공급시설,가축사육제한구역,개발제한구역,개발진흥지구,개발행위허가제한지역,건축선,건축용도지역기타,건축용도지역미분류,건축허가·착공제한지역,경관광장,경관녹지,경관지구,경관지구기타,고가도로,고도지구,고등학교,고속철도,고압선,공간시설미분류,공공공지,공공도서관,공공문화체육시설미분류,공공시설구역,공공시설용지,공공주택지구,공공지원민간임대주택 공급촉진지구,공공청사,공동구,공설화장시설,공원,공원마을지구(공원집단시설지구),공원문화유산지구,공원자연보존지구,공원자연환경지구,공익용산지,공익임지,공장설립승인지역,공장설립제한지역,공항,공항소음피해예상지역,공항소음피해지역,과밀억제권역,광로1류(폭 70M 이상),광로2류(폭 50M~70M),광로3류(폭 40M~50M),광역계획구역,광역복합환승센터,광장,교육환경보호구역,교차점광장,교통광장,교통운수시설미분류,국가산업단지,국가지정문화재구역,국가하천,국립공원,국민임대주택단지예정지구,국지도로,국토이용기타용도지구,국토이용용도지구기타,군사기지 및 군사시설 보호구역,궤도,근린공원,근린광장,근린상업지역,기타공공공지시설,기타공공청사시설,기타공원시설,기타교통시설,기타녹지시설,기타도로시설,기타도시공간시설,기타도시방재시설,기타문화시설,기타방송통신시설,기타보건위생시설,기타사회복지시설,기타수도시설,기타시장시설,기타열공급설비,기타용도지역지구기타,기타용도지역지구미분류,기타용지,기타유통및공급시설,기타유통업무설비,기타자동차정류장,기타전기공급설비,기타주차장시설,기타철도시설,기타폐기물처리시설,기타하수도시설,기타학교시설,기타환경기초시설,노외주차장,녹지,농수산물공판장및농수산물종합유통센터,대공방어협조구역,대공방어협조구역(위탁고도:54-236m),대공방어협조구역(위탁고도:77-257m),대로1류(폭 35M~40M),대로2류(폭 30M~35M),대로3류(폭 25M~30M),대학,도로,도로구역,도서관,도시개발구역,도시개발구역기타,도시고속도로,도시관리계획 입안중,도시기타용도지역지구기타,도시기타용도지역지구미분류,도시기타용도지역지구용도지역지구,도시자연공원,도시자연공원구역,도시지역,도시지역기타,도시철도,등록문화재구역,묘지공원,문화공원,문화시설,문화재,문화재보존영향 검토대상구역,문화재보호구역,문화재보호구역기타,문화지구,박물관,방송통신시설,방수설비,방화지구,배수시설,변전소(전원개발사업구역),변전시설,보건위생시설미분류,보전녹지지역,보전산지,보전임지,보존지구,보행자전용도로,부설주차장,비오톱1등급,비행안전제1구역(전술),비행안전제2구역(전술),비행안전제2구역(지원),비행안전제3구역(전술),비행안전제3구역(지원),비행안전제4구역(전술),비행안전제4구역(지원),비행안전제5구역(전술),비행안전제5구역(지원),비행안전제6구역(전술),사고지,사방시설,사방지,사업지역기타,사회복지시설,산업개발진흥지구,산업기술단지,산업시설구역,상대보호구역,상수원보호구역,생산녹지지역,생태·경관보전지역,생태·경관완충보전구역,생태·경관핵심보전구역,소공원,소로1류(폭 10M~12M),소로2류(폭 8M~10M),소로3류(폭 8M 미만),소하천,소하천구역,소하천예정지,수도공급시설,수질오염방지시설,수평표면구역,시·도 생태·경관보전지역,시·도야생생물보호구역,시가지경관지구,시가지조성사업지역,시도지정문화재구역,시장,시장정비구역,시장정비구역기타,시험림구역,아파트지구,액화석유가스충전시설,야생생물보호구역,어린이공원,여객자동차터미널,역사도심,역사문화미관지구,역사문화특화경관지구,역사문화환경보존지역,연결녹지,연구개발특구,연구시설,열공급설비,온천공보호구역,온천원보호지구,완충녹지,용도구역기타,용도구역미분류,용도지구취락지구,운동장,원추표면구역,원형보존지,유류저장및송유설비,유수시설,유수지,유원지,유치원,유통단지,유통상업지역,유통업무설비,일단의공업용지조성사업지역,일단의주택단지조성사업지역,일반광장,일반도로,일반미관지구,일반산업단지,일반상업지역,일반주거지역,일반철도,임업용산지,자동차검사시설,자동차운전학원,자동차전용도로,자동차정류장,자연경관지구,자연공원용도지구기타,자연녹지지역,자연재해위험지구,장애물제한표면구역,재개발구역,재개발구역기타,재정비촉진지구,재정비촉진지구기타,재해위험지구기타,재활용시설,저류시설,전기공급설비,전용주거지역,전이표면구역,전통사찰보존구역,절대보호구역,정비구역,정비구역기타,정비예정구역,제1종일반주거지역,제1종전용주거지역,제1종지구단위계획구역,제2종일반주거지역,제2종전용주거지역,제3종 구역,제3종일반주거지역,제방,제한보호구역,제한보호구역(방공기지 : 1km),제한보호구역(전술항공:5km),제한보호구역(후방지역:500m),조례로정한지역,조망가로미관지구,조망가로특화경관지구,종합운동장,종합의료시설,주간선도로,주거용지,주거환경개선지구,주요시설광장,주차장,주차장기타,주차환경개선지구,준공업지역,준보전산지,준주거지역,중로1류(폭 20M~25M),중로2류(폭 15M~20M),중로3류(폭 12M~15M),중심대광장,중심상업지역,중요시설물보호지구,중요시설물보호지구(공용),중요시설물보호지구(공항),중점경관관리구역,중학교,지구단위계획구역,지역특화발전특구,지원시설구역,지정문화재구역,지하광장,지하도로,진입표면구역,집단취락지구,집산도로,철도,청사,청소년수련시설,체육공원,체육시설,초등학교,취수시설,침수위험지구,택지개발예정지구,택지개발예정지구기타,토지거래계약에관한허가구역,토지구획정리사업지구기타,토지형질변경규제지역,통제보호구역,통제보호구역(민통선이남:300m),통제보호구역(방공기지:500m),특수도로(보행자전용도로),특정개발진흥지구,특화경관지구,폐기물처리및재활용시설,하수도,하수종말처리시설,하천,하천구역,하천미분류,학교,학교이적지,현상변경허가 대상구역,홍수관리구역,환경정비구역
0,서울특별시 강남구 개포동 1163-4,토지대장,2006,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,포함,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,접함,해당없음,포함,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,접함,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,포함,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,포함,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,

In [15]:
completed_df.to_csv('land_plans_ver1.csv', index=False)

In [16]:
completed_df.head(50)

,지번주소,대장구분명,년,(한강)오염행위 제한지역,(한강)폐기물매립시설 설치제한지역,가로구역별 최고높이 제한지역,가스공급설비,가스공급시설,가축사육제한구역,개발제한구역,개발진흥지구,개발행위허가제한지역,건축선,건축용도지역기타,건축용도지역미분류,건축허가·착공제한지역,경관광장,경관녹지,경관지구,경관지구기타,고가도로,고도지구,고등학교,고속철도,고압선,공간시설미분류,공공공지,공공도서관,공공문화체육시설미분류,공공시설구역,공공시설용지,공공주택지구,공공지원민간임대주택 공급촉진지구,공공청사,공동구,공설화장시설,공원,공원마을지구(공원집단시설지구),공원문화유산지구,공원자연보존지구,공원자연환경지구,공익용산지,공익임지,공장설립승인지역,공장설립제한지역,공항,공항소음피해예상지역,공항소음피해지역,과밀억제권역,광로1류(폭 70M 이상),광로2류(폭 50M~70M),광로3류(폭 40M~50M),광역계획구역,광역복합환승센터,광장,교육환경보호구역,교차점광장,교통광장,교통운수시설미분류,국가산업단지,국가지정문화재구역,국가하천,국립공원,국민임대주택단지예정지구,국지도로,국토이용기타용도지구,국토이용용도지구기타,군사기지 및 군사시설 보호구역,궤도,근린공원,근린광장,근린상업지역,기타공공공지시설,기타공공청사시설,기타공원시설,기타교통시설,기타녹지시설,기타도로시설,기타도시공간시설,기타도시방재시설,기타문화시설,기타방송통신시설,기타보건위생시설,기타사회복지시설,기타수도시설,기타시장시설,기타열공급설비,기타용도지역지구기타,기타용도지역지구미분류,기타용지,기타유통및공급시설,기타유통업무설비,기타자동차정류장,기타전기공급설비,기타주차장시설,기타철도시설,기타폐기물처리시설,기타하수도시설,기타학교시설,기타환경기초시설,노외주차장,녹지,농수산물공판장및농수산물종합유통센터,대공방어협조구역,대공방어협조구역(위탁고도:54-236m),대공방어협조구역(위탁고도:77-257m),대로1류(폭 35M~40M),대로2류(폭 30M~35M),대로3류(폭 25M~30M),대학,도로,도로구역,도서관,도시개발구역,도시개발구역기타,도시고속도로,도시관리계획 입안중,도시기타용도지역지구기타,도시기타용도지역지구미분류,도시기타용도지역지구용도지역지구,도시자연공원,도시자연공원구역,도시지역,도시지역기타,도시철도,등록문화재구역,묘지공원,문화공원,문화시설,문화재,문화재보존영향 검토대상구역,문화재보호구역,문화재보호구역기타,문화지구,박물관,방송통신시설,방수설비,방화지구,배수시설,변전소(전원개발사업구역),변전시설,보건위생시설미분류,보전녹지지역,보전산지,보전임지,보존지구,보행자전용도로,부설주차장,비오톱1등급,비행안전제1구역(전술),비행안전제2구역(전술),비행안전제2구역(지원),비행안전제3구역(전술),비행안전제3구역(지원),비행안전제4구역(전술),비행안전제4구역(지원),비행안전제5구역(전술),비행안전제5구역(지원),비행안전제6구역(전술),사고지,사방시설,사방지,사업지역기타,사회복지시설,산업개발진흥지구,산업기술단지,산업시설구역,상대보호구역,상수원보호구역,생산녹지지역,생태·경관보전지역,생태·경관완충보전구역,생태·경관핵심보전구역,소공원,소로1류(폭 10M~12M),소로2류(폭 8M~10M),소로3류(폭 8M 미만),소하천,소하천구역,소하천예정지,수도공급시설,수질오염방지시설,수평표면구역,시·도 생태·경관보전지역,시·도야생생물보호구역,시가지경관지구,시가지조성사업지역,시도지정문화재구역,시장,시장정비구역,시장정비구역기타,시험림구역,아파트지구,액화석유가스충전시설,야생생물보호구역,어린이공원,여객자동차터미널,역사도심,역사문화미관지구,역사문화특화경관지구,역사문화환경보존지역,연결녹지,연구개발특구,연구시설,열공급설비,온천공보호구역,온천원보호지구,완충녹지,용도구역기타,용도구역미분류,용도지구취락지구,운동장,원추표면구역,원형보존지,유류저장및송유설비,유수시설,유수지,유원지,유치원,유통단지,유통상업지역,유통업무설비,일단의공업용지조성사업지역,일단의주택단지조성사업지역,일반광장,일반도로,일반미관지구,일반산업단지,일반상업지역,일반주거지역,일반철도,임업용산지,자동차검사시설,자동차운전학원,자동차전용도로,자동차정류장,자연경관지구,자연공원용도지구기타,자연녹지지역,자연재해위험지구,장애물제한표면구역,재개발구역,재개발구역기타,재정비촉진지구,재정비촉진지구기타,재해위험지구기타,재활용시설,저류시설,전기공급설비,전용주거지역,전이표면구역,전통사찰보존구역,절대보호구역,정비구역,정비구역기타,정비예정구역,제1종일반주거지역,제1종전용주거지역,제1종지구단위계획구역,제2종일반주거지역,제2종전용주거지역,제3종 구역,제3종일반주거지역,제방,제한보호구역,제한보호구역(방공기지 : 1km),제한보호구역(전술항공:5km),제한보호구역(후방지역:500m),조례로정한지역,조망가로미관지구,조망가로특화경관지구,종합운동장,종합의료시설,주간선도로,주거용지,주거환경개선지구,주요시설광장,주차장,주차장기타,주차환경개선지구,준공업지역,준보전산지,준주거지역,중로1류(폭 20M~25M),중로2류(폭 15M~20M),중로3류(폭 12M~15M),중심대광장,중심상업지역,중요시설물보호지구,중요시설물보호지구(공용),중요시설물보호지구(공항),중점경관관리구역,중학교,지구단위계획구역,지역특화발전특구,지원시설구역,지정문화재구역,지하광장,지하도로,진입표면구역,집단취락지구,집산도로,철도,청사,청소년수련시설,체육공원,체육시설,초등학교,취수시설,침수위험지구,택지개발예정지구,택지개발예정지구기타,토지거래계약에관한허가구역,토지구획정리사업지구기타,토지형질변경규제지역,통제보호구역,통제보호구역(민통선이남:300m),통제보호구역(방공기지:500m),특수도로(보행자전용도로),특정개발진흥지구,특화경관지구,폐기물처리및재활용시설,하수도,하수종말처리시설,하천,하천구역,하천미분류,학교,학교이적지,현상변경허가 대상구역,홍수관리구역,환경정비구역
0,서울특별시 강남구 개포동 1163-4,토지대장,2006,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,포함,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,접함,해당없음,포함,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,접함,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,포함,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,포함,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,해당없음,

In [13]:
def create_complete_land_plans_df_reshape(df):
    df = df.copy()
    gc.collect()
    
    print('1')
    df = df.set_index(['지번주소', '대장구분명', '년']).stack(dropna=False)
    print('2')
    df = df.reset_index()
    print('3')
    df['binary'] = 1
    print('4')
    df = df.set_index(['지번주소', '대장구분명', 'level_3', 0, '년']).unstack()
    print('5')
    df[('binary', '2006')] = np.nan
    print('6')
    df = df.stack(dropna=False)
    print('7')
    df = df.reset_index()
    print('8')
    df = df[['지번주소', '대장구분명', 'level_3', 0, '년']]
    print('9')
    df = df.set_index(['지번주소', '대장구분명', '년', 'level_3']).unstack()
    print('10')
    
    return df

In [ ]:
# decided not to use it due to its super high memory consumption
#prac_df = create_complete_land_plans_df_reshape(selected_land_plans_df)